In [1]:
from pydantic import BaseModel, Field
from typing import List
import guidance
import os


In [2]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [2]:
base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Instruct-2507", base_url="http://127.0.0.1:30000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [3]:
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print(lm["answer"])

# run_gen_test(base_lm)

In [4]:
from pydantic import BaseModel, Field
from typing import List
import guidance
from guidance import models, system, user, assistant, json as gen_json, select
import json
from enum import Enum


class FirstPrinciples(BaseModel):
    absolute_truths: List[str] = Field(
        ...,
        description="Look at the problem statement and list the things that you know that are absolute truths."
    )


class GroundUpReasoning(BaseModel):
    """Model for the LLM-generated reasoning plan."""
    reason: str = Field(
        ...,
        description="Based on the known truths reason in a concise sharp manner from ground up to address the problem."
    )

class Reflection(BaseModel):
    """Model for an individual reasoning step execution."""
    evaluation: str = Field(
        ...,
        description="Reflect on your reasoning and identify gaps / uncertainties / mistakes."
    )

class ReasoningConclusion(BaseModel):
    """Model for the final reasoning conclusion."""
    conclusion: str = Field(
        ...,
        description="Address the identified gaps in the previous step if any and deduce the final conclusion."
    )

def dynamic_final_answer_schema(Options):
    class FinalAnswer(BaseModel):
        answer: Options = Field(..., description="Final answer.")

    return FinalAnswer


@guidance()
def _first_principles(llm):
    with assistant():
        llm += gen_json(
            name="first_principles",
            schema=FirstPrinciples,
            max_tokens=5000
        )
    return llm

@guidance()
def _generate_groundup_reasoning(llm):
    """
    Generate a custom reasoning plan tailored to the query.
    """
    with assistant():
        llm += gen_json(
            name="groundup_reasoning",
            schema=GroundUpReasoning,
            max_tokens=5000  # Reduced to encourage conciseness
        )
    return llm

@guidance()
def _reflection(llm):
    with assistant():
        llm += gen_json(
            name="reflection",
            schema=Reflection,
            max_tokens=5000
        )
    return llm
    

@guidance()
def _reasoning_conclusion(llm):
    with user():
        llm += """Based on the information, make logical deductions."""
    
    with assistant():
        llm += gen_json(
            name="reasoning_conclusion",
            schema=ReasoningConclusion,
            max_tokens=5000
        )

    return llm

@guidance
def dynamic_structured_reasoning(llm, query, output_choices):
    Options = Enum("Choices", {choice: choice for choice in output_choices})
    final_answer_schema = dynamic_final_answer_schema(Options)

    with system():
        llm += """You are an expert in problem solving. For any query, reason from first principles to solve the query. 
        Structure all outputs as JSON. Be very brief in your steps, tending towards concise but complete responses.
        """

    with user():
        llm += f"""Query: {query}
        Look at the problem statement and list the things that you know that are absolute truths -
        """

    llm += _first_principles()

    with user():
        llm += f"""
        Based on the known truths reason from ground up to address the problem, be extremely brief and to the point in your reasoning -
        """

    llm += _generate_groundup_reasoning()

    with user():
        llm += f"""
        Reflect to identify gaps / uncertainties / mistakes in reasoning process -
        """

    llm += _reflection()

    with user():
        llm += f"""
        Make logical deductions based on the information you have to arrive at a conclusion -
        """

    llm += _reasoning_conclusion()

    with user():
        llm += f"""
        Your final answer -
        """

    with assistant():
        llm += gen_json(
            name="answer",
            schema=final_answer_schema,
            max_tokens=1000)

    return llm

In [5]:
def generate(query, output_choices, lm = None):
    lm = lm or base_lm
    result_lm = lm + dynamic_structured_reasoning(
                    query=query,
                    output_choices = output_choices
                )

    answer = json.loads(result_lm.get("answer"))['answer']
    
    groundup_reasoning = json.loads(result_lm.get("groundup_reasoning"))['reason']
    first_principles = json.loads(result_lm.get("first_principles"))['absolute_truths']
    reflection = json.loads(result_lm.get("reflection"))['evaluation']   
    reasoning_conclusion = json.loads(result_lm.get("reasoning_conclusion"))['conclusion']
    return first_principles, groundup_reasoning, reflection, reasoning_conclusion, answer


In [6]:
from datasets import load_dataset

# Load the BBH dataset from Hugging Face
dataset = load_dataset("WildEval/ZebraLogic", 'mc_mode')



In [7]:
from json import JSONDecodeError


def evaluate_reasoning(row):
    """
    Placeholder function to evaluate your reasoning system on a single row.
    Replace with your dynamic_structured_reasoning function.
    """
    
    # from guidance.models.experimental import SglangModel
    # lm = SglangModel(
    #     model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    #     base_url="http://127.0.0.1:30000/v1",
    #     echo=True,
    #     api_key=os.environ.get("OPENAI_API_KEY", "NO_KEY"),
    # )
    puzzle = row['puzzle']
    question = row['question']
    query = puzzle + '\n\n' + question
    choices = row['choices']
    target = row['answer']
    print(row['id'])
    # Your reasoning system should return a predicted answer
    try:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = generate(query, choices)
    except JSONDecodeError as e:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = ["JSONDecodeError"]*5
    return {
        'correct': pred == target,
        'first_principles': first_principles,
        'groundup_reasoning': groundup_reasoning,
        'reflection': reflection,
        'reasoning_conclusion': reasoning_conclusion,
        'predicted_answer': pred, 
    }


In [8]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import pandas as pd

# # df is already df.iloc[600:650]  (keeps original index)
# def run_parallel_preserve_index(df, workers=16):
#     idx_to_row = df.to_dict(orient="index")  # {index: rowdict}
#     results = {}

#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         futs = {ex.submit(evaluate_reasoning, row): idx
#                 for idx, row in idx_to_row.items()}
#         for fut in as_completed(futs):
#             idx = futs[fut]
#             results[idx] = fut.result()

#     res_df = pd.DataFrame.from_dict(results, orient="index")
#     # ensure order matches df (and keep original index)
#     res_df = res_df.reindex(df.index)
#     return df.join(res_df)

# df = pd.DataFrame(dataset['test'])
# # df.head()
# # CURRENTLY FINISHED - [:600]

# df = df.iloc[600:605]
# df = run_parallel_preserve_index(df, workers=16)


In [9]:
import pandas as pd
df = pd.DataFrame(dataset['test'])
len(df)

3259

In [10]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
# df.head()
# CURRENTLY FINISHED - [:3100]

df = df.iloc[3100:3260]

res = df.apply(lambda r: pd.Series(evaluate_reasoning(r)), axis=1)
df = pd.concat([df, res], axis=1)


lgp-test-6x5-2#mc-16


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lgp-test-5x4-3#mc-3
lgp-test-3x6-34#mc-14
lgp-test-6x5-13#mc-20
lgp-test-6x6-17#mc-2
lgp-test-3x2-25#mc-0
lgp-test-6x2-38#mc-11
lgp-test-2x6-11#mc-5
lgp-test-4x6-36#mc-22
lgp-test-2x6-7#mc-5
lgp-test-6x6-34#mc-13
lgp-test-2x5-27#mc-7
lgp-test-4x5-34#mc-17
lgp-test-5x5-10#mc-23
lgp-test-2x2-35#mc-0
lgp-test-5x5-9#mc-18
lgp-test-4x2-33#mc-6
lgp-test-3x2-37#mc-4
lgp-test-6x5-6#mc-22
lgp-test-3x5-28#mc-9
lgp-test-5x6-21#mc-28
lgp-test-5x6-37#mc-19
lgp-test-2x3-24#mc-2
lgp-test-2x6-15#mc-0
lgp-test-5x3-5#mc-7
lgp-test-6x4-33#mc-23
lgp-test-4x6-19#mc-19
lgp-test-3x6-31#mc-5
lgp-test-2x6-25#mc-0
lgp-test-5x5-8#mc-2
lgp-test-3x6-14#mc-3
lgp-test-3x4-27#mc-2
lgp-test-2x4-18#mc-4
lgp-test-6x6-12#mc-22
lgp-test-4x6-22#mc-14
lgp-test-5x6-34#mc-3
lgp-test-5x3-27#mc-13
lgp-test-6x6-10#mc-9
lgp-test-5x6-32#mc-10
lgp-test-4x4-27#mc-3
lgp-test-5x4-13#mc-12
lgp-test-3x4-7#mc-0
lgp-test-2x4-22#mc-1
lgp-test-4x5-39#mc-15
lgp-test-4x6-37#mc-7
lgp-test-5x5-1#mc-17
lgp-test-6x3-14#mc-16
lgp-test-2x2-2#mc-1
l

In [11]:
df

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
3100,lgp-test-6x5-2#mc-16,"There are 6 houses, numbered 1 to 6 from left ...",What is Mother of the person who lives in Hous...,"[Janelle, Holly, Aniya, Sarah, Kailyn, Penny]",Penny,2024-07-03T21:21:31.376158,True,[House 4 has the person who loves cultural tou...,House 4 has cultural tours (Clue 9) and myster...,The reasoning correctly identifies that House ...,The mother of the person in House 4 is Penny.,Penny
3101,lgp-test-5x4-3#mc-3,"There are 5 houses, numbered 1 to 5 from left ...",What is Food of the person who lives in House 1?,"[pizza, spaghetti, stew, stir fry, grilled che...",pizza,2024-07-03T21:21:31.343515,False,"[Arnold loves pop music (Clue 12)., The person...",Arnold → pop → mother Kailyn. Kailyn is left o...,The reasoning assumes that 'grilled cheese' is...,House 1: Food = grilled cheese,grilled cheese
3102,lgp-test-3x6-34#mc-14,"There are 3 houses, numbered 1 to 3 from left ...",What is BookGenre of the person who lives in H...,"[science fiction, mystery, romance]",mystery,2024-07-03T21:21:31.318007,True,[Peter's birthday is in September (from clue 3...,Peter has Sept birthday → owns Toyota Camry. J...,The reasoning correctly deduces that house 3 c...,The book genre of the person in House 3 is mys...,mystery
3103,lgp-test-6x5-13#mc-20,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 5?,"[Peter, Eric, Arnold, Alice, Bob, Carol]",Alice,2024-07-03T21:21:31.378465,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
3104,lgp-test-6x6-17#mc-2,"There are 6 houses, numbered 1 to 6 from left ...",What is Food of the person who lives in House 1?,"[pizza, grilled cheese, spaghetti, stir fry, s...",soup,2024-07-03T21:21:31.388872,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,lgp-test-5x2-12#mc-7,"There are 5 houses, numbered 1 to 5 from left ...",What is Smoothie of the person who lives in Ho...,"[cherry, lime, watermelon, desert, dragonfruit]",watermelon,2024-07-03T21:21:31.338248,True,"[Arnold is in the third house., The person who...",Arnold is in house 3 and drinks Lime smoothies...,Rechecking: Arnold in house 3 → Lime in house ...,The smoothie of the person in House 4 is Water...,watermelon
3255,lgp-test-5x6-10#mc-15,"There are 5 houses, numbered 1 to 5 from left ...",What is Smoothie of the person who lives in Ho...,"[desert, watermelon, cherry, lime, dragonfruit]",cherry,2024-07-03T21:21:31.356417,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
3256,lgp-test-6x4-23#mc-20,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 6?,"[Eric, Alice, Arnold, Carol, Bob, Peter]",Arnold,2024-07-03T21:21:31.373097,True,"[House 3 has the bird keeper (Clue 1)., The pe...","Peter is in House 3 (Clues 6, 15). Eric is Des...",The reasoning is logically consistent and full...,The person in House 6 is Arnold.,Arnold
3257,lgp-test-4x6-28#mc-23,"There are 4 houses, numbered 1 to 4 from left ...",What is Drink of the person who lives in House 4?,"[tea, coffee, milk, water]",tea,2024-07-03T21:21:31.335835,True,[House 2 has the person who loves mystery book...,"House 2: mystery books, September birthday, wa...",The reasoning process is logically sound and c...,The drink of the person in House 4 is tea.,tea


In [12]:
df['correct'].value_counts()

correct
True     111
False     48
Name: count, dtype: int64

In [13]:
path = './zebralogic_eval.csv'
df.to_csv(path, mode="a", index=False, header=not os.path.exists(path))

In [20]:
import pandas as pd
path = './zebralogic_eval.csv'
dub = pd.read_csv(path)


In [21]:
dub['correct'].value_counts()

correct
True     2365
False     894
Name: count, dtype: int64

In [22]:
dub

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
0,lgp-test-6x4-37#mc-16,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 5?,"['Eric', 'Bob', 'Alice', 'Peter', 'Carol', 'Ar...",Bob,2024-07-03T21:21:31.375234,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
1,lgp-test-3x6-15#mc-13,"There are 3 houses, numbered 1 to 3 from left ...",What is Education of the person who lives in H...,"['associate', 'high school', 'bachelor']",associate,2024-07-03T21:21:31.316147,True,"['Peter is in House 1.', 'Eric is in House 3.'...",Peter (House 1) has bachelor's degree → House ...,Initial reasoning correctly deduces: House 1: ...,House 3 has education: associate.,associate
2,lgp-test-4x6-24#mc-9,"There are 4 houses, numbered 1 to 4 from left ...",What is BookGenre of the person who lives in H...,"['science fiction', 'mystery', 'romance', 'fan...",mystery,2024-07-03T21:21:31.335059,True,"['House 1 has the bird keeper (Clue 2).', 'The...","House 1: red hair (Clue 6), bird (Clue 2). Hou...",Initial reasoning incorrectly assumed black ha...,The BookGenre of the person in House 2 is myst...,mystery
3,lgp-test-5x5-18#mc-0,"There are 5 houses, numbered 1 to 5 from left ...",What is Name of the person who lives in House 1?,"['Peter', 'Bob', 'Arnold', 'Eric', 'Alice']",Eric,2024-07-03T21:21:31.350746,True,['House 1 is occupied by the teacher (Clue 8)....,House 1 is the teacher (Clue 8). The only name...,"The reasoning correctly eliminates Bob, Alice,...",The person in House 1 is Eric.,Eric
4,lgp-test-4x2-35#mc-0,"There are 4 houses, numbered 1 to 4 from left ...",What is Name of the person who lives in House 1?,"['Eric', 'Arnold', 'Alice', 'Peter']",Peter,2024-07-03T21:21:31.319814,True,"['Eric is in the second house.', 'The Swedish ...","Alice is in house 3 (clue 5), so she is Britis...",The reasoning assumes Arnold is not in house 1...,The person in House 1 is Peter.,Peter
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,lgp-test-5x2-12#mc-7,"There are 5 houses, numbered 1 to 5 from left ...",What is Smoothie of the person who lives in Ho...,"['cherry', 'lime', 'watermelon', 'desert', 'dr...",watermelon,2024-07-03T21:21:31.338248,True,"['Arnold is in the third house.', 'The person ...",Arnold is in house 3 and drinks Lime smoothies...,Rechecking: Arnold in house 3 → Lime in house ...,The smoothie of the person in House 4 is Water...,watermelon
3255,lgp-test-5x6-10#mc-15,"There are 5 houses, numbered 1 to 5 from left ...",What is Smoothie of the person who lives in Ho...,"['desert', 'watermelon', 'cherry', 'lime', 'dr...",cherry,2024-07-03T21:21:31.356417,False,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError,JSONDecodeError
3256,lgp-test-6x4-23#mc-20,"There are 6 houses, numbered 1 to 6 from left ...",What is Name of the person who lives in House 6?,"['Eric', 'Alice', 'Arnold', 'Carol', 'Bob', 'P...",Arnold,2024-07-03T21:21:31.373097,True,"['House 3 has the bird keeper (Clue 1).', 'The...","Peter is in House 3 (Clues 6, 15). Eric is Des...",The reasoning is logically consistent and full...,The person in House 6 is Arnold.,Arnold
3257,lgp-test-4x6-28#mc-23,"There are 4 houses, numbered 1 to 4 from left ...",What is Drink of the person who lives in House 4?,"['tea', 'coffee', 'milk', 'water']",tea,2024-07-03T21:21:31.335835,True,['House 2 has the person who loves mystery boo...,"House 2: mystery books, September birthday, wa...",The reasoning process is logically sound and c...,The drink of the person in House 4 is tea.,tea
